# RAG

Retrieval-Augmented Generation (RAG) is a technique that combines the strengths of pre-trained language models with the ability to retrieve information from a large corpus of documents. RAG **enables the language model to produce more informed, accurate, and contextually relevant answers** than by relying on its pre-trained knowledge alone.

At Cohere, all RAG calls come with... **precise citations**! 🎉
The model cites which groups of words, in the RAG chunks, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from and they help users gain visibility into the model reasoning.  

RAG consists of 3 steps:
- Step 1: Indexing and given a user query, retrieve the relevant chunks from the index
- Step 2: Optionally, rerank the retrieved chunks
- Step 3: Generate the model final answer with **precise citations**, given the retrieved and reranked chunks




## Step 0 - Imports & Getting some data

In this example, we'll use a recent piece of text, that wasn't in the training data: the Wikipedia page of the movie "Dune 2".   

In practice, you would typically do RAG on much longer text, that doesn't fit in the context window of the model.

In [1]:
# pip install cohere

import cohere

co = cohere.ClientV2(api_key="YOUR_COHERE_API_KEY") # Get your free API key: https://dashboard.cohere.com/api-keys

In [3]:
# we'll get some wikipedia data
# ! pip install wikipedia -qq

import wikipedia

In [4]:
# let's get the wikipedia article about Dune Part Two
article = wikipedia.page('Dune Part Two')
text = article.content
print(f"The text has roughly {len(text.split())} words.")

The text has roughly 5896 words.


## Step 1 - Indexing and given a user query, retrieve the relevant chunks from the index

We index the document in a vector database. This requires getting the documents, chunking them, embedding, and indexing them in a vector database. Then we retrieved relevant results based on the users' query.

### We split the document into chunks of roughly 512 words

In [5]:
# For chunking let's use langchain to help us split the text
! pip install -qU langchain-text-splitters -qq

from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
# Create basic configurations to chunk the text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# Split the text into chunks with some overlap
chunks_ = text_splitter.create_documents([text])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 108 chunks.


### Embed every text chunk

Cohere embeddings are state-of-the-art.

In [8]:
# Because the texts being embedded are the chunks we are searching over, we set the input type as search_doc
model = "embed-english-v3.0"

def batch_embed(texts, batch_size=96):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = co.embed(
            texts=batch,
            model=model,
            input_type="search_document",
            embedding_types=['float']
        )
        all_embeddings.extend(response.embeddings.float)
    return all_embeddings

embeddings = batch_embed(chunks)
print(f"We just computed {len(embeddings)} embeddings.")

We just computed 108 embeddings.


### Store the embeddings in a vector database

We use the simplest vector database ever: a python dictionary using `np.array()`.

In [10]:
# We use the simplest vector database ever: a python dictionary
! pip install numpy -qq

In [9]:
import numpy as np
vector_database = {i: np.array(embedding) for i, embedding in enumerate(embeddings)}
# { 0: array([...]), 1: array([...]), 2: array([...]), ..., 10: array([...]) }

## Given a user query, retrieve the relevant chunks from the vector database


### Define the user question

In [10]:
query = "Name everyone involved in writing the script, directing, and producing 'Dune: Part Two'?"

# Note: the relevant passage in the wikipedia page we're looking for is:
# "[...] Dune: Part Two was originally scheduled to be released on October 20, 2023, but was delayed to November 17, 2023, before moving forward two weeks to November 3, 2023, to adjust to changes in release schedules from other studios. It was later postponed by over four months to March 15, 2024, due to the 2023 Hollywood labor disputes. After the strikes were resolved, the film moved once more up two weeks to March 1, 2024. [...]"

### Embed the user question

Cohere embeddings are state-of-the-art.

In [11]:
# Because the text being embedded is the search query, we set the input type as search_query
response = co.embed(
    texts=[query],
    model=model,
    input_type="search_query",
    embedding_types=['float']
)
query_embedding = response.embeddings.float[0]
print("query_embedding: ", query_embedding[:10] + ["..."])

query_embedding:  [-0.068603516, -0.02947998, -0.06274414, -0.015449524, -0.033294678, 0.0056877136, -0.047210693, 0.04714966, -0.024871826, 0.008148193, '...']


### Retrieve the most relevant chunks from the vector database

We use cosine similarity to find the most similar chunks

In [12]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calculate similarity between the user question & each chunk
similarities = [cosine_similarity(query_embedding, chunk) for chunk in embeddings]
print("similarity scores: ", similarities)

# Get indices of the top 10 most similar chunks
sorted_indices = np.argsort(similarities)[::-1]

# Keep only the top 10 indices
top_indices = sorted_indices[:10]
print("Here are the indices of the top 10 chunks after retrieval: ", top_indices)

# Retrieve the top 10 most similar chunks
top_chunks_after_retrieval = [chunks[i] for i in top_indices]
print("Here are the top 10 chunks after retrieval: ")
for t in top_chunks_after_retrieval:
    print("== " + t)

similarity scores:  [0.6880419226352857, 0.3923392545434105, 0.6386815248269733, 0.43618145115797535, 0.35780784367307705, 0.3923210354894715, 0.3406860631010847, 0.2957357005793907, 0.4255159555931599, 0.14503223489330883, 0.3997846789722547, 0.3743933002525113, 0.4083288113267294, 0.26194266616864986, 0.31982912150076953, 0.3531164824914523, 0.23203650717727414, 0.497148799385201, 0.34487158221487835, 0.28824423308146263, 0.5782290303802001, 0.5489319961327446, 0.778415400224538, 0.5236541350938922, 0.5481962732642285, 0.7135549552247115, 0.5205999648021981, 0.5872268968480292, 0.26489409375108064, 0.6410858426784221, 0.5367759890829636, 0.6816158875713925, 0.39144361423311, 0.4827300300834518, 0.4502725966061075, 0.2462873309174519, 0.443300705486635, 0.39120172781317253, 0.20620678706300988, 0.4371599199261209, 0.37572638119299867, 0.4614621126654143, 0.29906070488176123, 0.36477353501504023, 0.36024747243912564, 0.3929914083901088, 0.23114430973151348, 0.4646991530089856, 0.118742

## Step 2 - Rerank the chunks retrieved from the vector database

We rerank the 10 chunks retrieved from the vector database. Reranking boosts retrieval accuracy.

Reranking lets us go from 10 chunks retrieved from the vector database, to the 3 most relevant chunks.

In [13]:
response = co.rerank(
    query=query,
    documents=top_chunks_after_retrieval,
    top_n=3,
    model="rerank-english-v3.0",
)

# top_chunks_after_rerank = [result.document['text'] for result in response]

top_chunks_after_rerank = [top_chunks_after_retrieval[result.index] for result in response.results]

print("Here are the top 3 chunks after rerank: ")
for t in top_chunks_after_rerank:
    print("== " + t)

Here are the top 3 chunks after rerank: 
== Dune: Part Two was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tanya Lapointe, Brian Herbert, Byron Merritt, Kim Herbert, Thomas Tull, Jon Spaihts, Richard P. Rubinstein, John Harrison, and Herbert W. Gain serving as executive producers and Kevin J. Anderson as creative consultant. Legendary CEO Joshua Grode confirmed in April 2019 that they plan to make a sequel, adding that "there's a logical place to stop the [first] movie before the book is over".
== Dune: Part Two is a 2024 American epic science fiction film directed and co-produced by Denis Villeneuve, who co-wrote the screenplay with Jon Spaihts. The sequel to Dune (2021), it is the second of a two-part adaptation of the 1965 novel Dune by Frank Herbert. It follows Paul Atreides as he unites with the Fremen people of the desert planet Arrakis to wage war against House Harkonnen. Timothée Chalamet, Rebecca Ferguson, Josh Brolin, Stellan Skarsgård, Dave Bautista, Zendaya, 

## Step 3 - Generate the model final answer, given the retrieved and reranked chunks

In [14]:
# preamble containing instructions about the task and the desired style for the output.
preamble = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

In [16]:
# retrieved documents
documents = [
    {"data": {"title": "chunk 0", "snippet": top_chunks_after_rerank[0]}},
    {"data": {"title": "chunk 1", "snippet": top_chunks_after_rerank[1]}},
    {"data": {"title": "chunk 2", "snippet": top_chunks_after_rerank[2]}},
  ]

# get model response
response = co.chat(
  model="command-r-08-2024",
  messages=[{"role" : "system", "content" : preamble},
            {"role" : "user", "content" : query}],
  documents=documents,  
  temperature=0.3
)

print("Final answer:")
print(response.message.content[0].text)

Final answer:
*Dune: Part Two* is a 2024 American epic science fiction film directed and co-produced by Denis Villeneuve, who co-wrote the screenplay with Jon Spaihts. The film was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tanya Lapointe, Brian Herbert, Byron Merritt, Kim Herbert, Thomas Tull, Richard P. Rubinstein, John Harrison, and Herbert W. Gain serving as executive producers. Kevin J. Anderson was the creative consultant.


Note: this is indeed the answer you'd expect, and here was the passage of text in wikipedia explaining it!

" [...] Dune: Part Two was originally scheduled to be released on October 20, 2023, but was delayed to November 17, 2023, before moving forward two weeks to November 3, 2023, to adjust to changes in release schedules from other studios. It was later postponed by over four months to March 15, 2024, due to the 2023 Hollywood labor disputes. After the strikes were resolved, the film moved once more up two weeks to March 1, 2024. [...]"

## Bonus: Citations come for free with Cohere! 🎉

At Cohere, all RAG calls come with... precise citations! 🎉
The model cites which groups of words, in the RAG chunks, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from.  
They help users gain visibility into the model reasoning, and sanity check the final model generation.  
These citations are optional — you can decide to ignore them.


In [18]:
print("Citations that support the final answer:")
for cite in response.message.citations:
    print(cite)

Citations that support the final answer:
start=22 end=61 text='2024 American epic science fiction film' sources=[DocumentSource(type='document', id='doc:1', document={'id': 'doc:1', 'snippet': 'Dune: Part Two is a 2024 American epic science fiction film directed and co-produced by Denis Villeneuve, who co-wrote the screenplay with Jon Spaihts. The sequel to Dune (2021), it is the second of a two-part adaptation of the 1965 novel Dune by Frank Herbert. It follows Paul Atreides as he unites with the Fremen people of the desert planet Arrakis to wage war against House Harkonnen. Timothée Chalamet, Rebecca Ferguson, Josh Brolin, Stellan Skarsgård, Dave Bautista, Zendaya, Charlotte Rampling, and Javier', 'title': 'chunk 1'})]
start=62 end=106 text='directed and co-produced by Denis Villeneuve' sources=[DocumentSource(type='document', id='doc:1', document={'id': 'doc:1', 'snippet': 'Dune: Part Two is a 2024 American epic science fiction film directed and co-produced by Denis Villeneuve, who 

In [23]:
def insert_inline_citations(text, citations, field='text'):
    sorted_citations = sorted(citations, key=lambda c: c.start, reverse=True)
    
    for citation in sorted_citations:
        source_ids = [source.id.split(':')[-1] for source in citation.sources]
        citation_text = f"[{','.join(source_ids)}]"
        text = text[:citation.end] + citation_text + text[citation.end:]
    
    return text

def list_sources(citations, fields=['text']):
    unique_sources = set()
    for citation in citations:
        for source in citation.sources:
            source_data = tuple((field, source.document[field]) for field in fields if field in source.document)
            unique_sources.add((source.id.split(':')[-1], source_data))
    
    footnotes = []
    for source_id, source_data in sorted(unique_sources):
        footnote = f"[{source_id}] " + ", ".join(f"{key}: {value}" for key, value in source_data)
        footnotes.append(footnote)
    
    return "\n".join(footnotes)

# Use the functions
cited_text = insert_inline_citations(response.message.content[0].text, response.message.citations)

# Print the result with inline citations
print(cited_text)

# Print footnotes
if response.message.citations:
    print("\nSource documents:")
    print(list_sources(response.message.citations, fields=['title','snippet']))

*Dune: Part Two* is a 2024 American epic science fiction film[1] directed and co-produced by Denis Villeneuve[1], who co-wrote the screenplay with Jon Spaihts.[1] The film was produced by Villeneuve[0], Mary Parent[0], and Cale Boyter[0], with Tanya Lapointe[0], Brian Herbert[0], Byron Merritt[0], Kim Herbert[0], Thomas Tull[0], Richard P. Rubinstein[0], John Harrison[0], and Herbert W. Gain[0] serving as executive producers.[0] Kevin J. Anderson[0] was the creative consultant.[0]

Source documents:
[0] title: chunk 0, snippet: Dune: Part Two was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tanya Lapointe, Brian Herbert, Byron Merritt, Kim Herbert, Thomas Tull, Jon Spaihts, Richard P. Rubinstein, John Harrison, and Herbert W. Gain serving as executive producers and Kevin J. Anderson as creative consultant. Legendary CEO Joshua Grode confirmed in April 2019 that they plan to make a sequel, adding that "there's a logical place to stop the [first] movie before the book is ov

In [24]:
def insert_inline_citations(text, citations, field='text'):
    sorted_citations = sorted(citations, key=lambda c: c.start, reverse=True)
    
    for citation in sorted_citations:
        source_ids = [source.id.split(':')[-1] for source in citation.sources]
        citation_text = f"[{','.join(source_ids)}]"
        text = text[:citation.end] + citation_text + text[citation.end:]
    
    return text

def list_sources(citations):
    unique_sources = set()
    for citation in citations:
        for source in citation.sources:
            source_data = tuple((key, value) for key, value in source.document.items() if key != 'id')
            unique_sources.add((source.id.split(':')[-1], source_data))
    
    footnotes = []
    for source_id, source_data in sorted(unique_sources):
        footnote = f"[{source_id}] " + ", ".join(f"{key}: {value}" for key, value in source_data)
        footnotes.append(footnote)
    
    return "\n".join(footnotes)

# Use the functions
cited_text = insert_inline_citations(response.message.content[0].text, response.message.citations)

# Print the result with inline citations
print(cited_text)

# Print footnotes
if response.message.citations:
    print("\nSource documents:")
    print(list_sources(response.message.citations))

*Dune: Part Two* is a 2024 American epic science fiction film[1] directed and co-produced by Denis Villeneuve[1], who co-wrote the screenplay with Jon Spaihts.[1] The film was produced by Villeneuve[0], Mary Parent[0], and Cale Boyter[0], with Tanya Lapointe[0], Brian Herbert[0], Byron Merritt[0], Kim Herbert[0], Thomas Tull[0], Richard P. Rubinstein[0], John Harrison[0], and Herbert W. Gain[0] serving as executive producers.[0] Kevin J. Anderson[0] was the creative consultant.[0]

Source documents:
[0] snippet: Dune: Part Two was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tanya Lapointe, Brian Herbert, Byron Merritt, Kim Herbert, Thomas Tull, Jon Spaihts, Richard P. Rubinstein, John Harrison, and Herbert W. Gain serving as executive producers and Kevin J. Anderson as creative consultant. Legendary CEO Joshua Grode confirmed in April 2019 that they plan to make a sequel, adding that "there's a logical place to stop the [first] movie before the book is over"., title: chu